# Fine Tune CLIP on Tweets

- CLIP on huggingface: https://huggingface.co/openai/clip-vit-base-patch32
- Dataset: https://huggingface.co/datasets/AlekseyDorkin/extended_tweet_emojis/tree/main

## 1. Install Dependencies

In [1]:
# you might want to restart the kernel
# coupling between torch and torchvision: https://pypi.org/project/torchvision/
!pip install torchvision==0.11.1 torch==1.10.0 --quiet

In [2]:
# you might want to restart the kernel after installation is complete.
!pip install transformers datasets pillow ipywidgets requests jupyter jupyter_client wandb --upgrade --quiet

## 2. Init Variables and Tools

In [3]:
pwd = !pwd
data_path = pwd[0] + "/emojis"
data_path

'/root/emoji-predictor/fine-tune/emojis'

In [4]:
import wandb
from transformers import TrainingArguments, Trainer

wandb.init(project="emoji-predictor", entity="drift-ai")

/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Curren

## 3. Setup Data Preprocessors and Trainer

In [45]:
from pathlib import Path

import torch
from torchvision.transforms import Resize, InterpolationMode, ConvertImageDtype, CenterCrop
from torchvision.io import read_image, ImageReadMode
from transformers import CLIPProcessor, CLIPModel, Trainer
from transformers import default_data_collator, TrainingArguments
from datasets import load_dataset, Dataset

# Loading Data
dataset = load_dataset("vincentclaes/emoji-predictor")
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]

column_names = train_dataset.column_names
assert "label" in column_names
assert "text" in column_names
image_column = "label"
caption_column = "text"


Using custom data configuration vincentclaes--emoji-predictor-84ee9ecf6ec78809
Reusing dataset parquet (/root/.cache/huggingface/datasets/vincentclaes___parquet/vincentclaes--emoji-predictor-84ee9ecf6ec78809/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [46]:
# train_dataset = train_dataset.train_test_split(test_size=0.05)
# train_dataset = train_dataset["test"]

In [47]:
# Loading Model and Processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
config = model.config
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = processor.tokenizer
feature_extractor = processor.feature_extractor

MAX_TEXT_LENGTH=77
IMAGE_SIZE = config.vision_config.image_size

# Preprocess Text
def tokenize_captions(examples):
    captions = [caption for caption in examples[caption_column]]
    text_inputs = tokenizer(captions, max_length=MAX_TEXT_LENGTH, padding="max_length", truncation=True)
    examples["input_ids"] = text_inputs.input_ids
    examples["attention_mask"] = text_inputs.attention_mask
    return examples


train_dataset = train_dataset.map(
    function=tokenize_captions,
    batched=True,
    remove_columns=[col for col in column_names if col != image_column],
    num_proc=None,
    load_from_cache_file=False,
    desc="Running tokenizer on train dataset",
)

val_dataset = val_dataset.map(
    function=tokenize_captions,
    batched=True,
    remove_columns=[col for col in column_names if col != image_column],
    num_proc=None,
    load_from_cache_file=False,
    desc="Running tokenizer on val dataset",
)

test_dataset = test_dataset.map(
    function=tokenize_captions,
    batched=True,
    remove_columns=[col for col in column_names if col != image_column],
    num_proc=None,
    load_from_cache_file=False,
    desc="Running tokenizer on test dataset",
)

# Preprocess Images
class Transform(torch.nn.Module):
    def __init__(self, image_size):
        super().__init__()
        self.transforms = torch.nn.Sequential(
            # resize and then crop the image to the image_size
            Resize([image_size], interpolation=InterpolationMode.BICUBIC),
            CenterCrop(image_size),
            # convert RGB to floats
            ConvertImageDtype(torch.float),
        )

    def forward(self, x) -> torch.Tensor:
        with torch.no_grad():
            x = self.transforms(x)
        return x


image_transformations = Transform(
    IMAGE_SIZE
)
image_transformations = torch.jit.script(image_transformations)


def transform_images(examples):
    # https://pytorch.org/vision/stable/_modules/torchvision/io/image.html#ImageReadMode
    images = [read_image(str(Path(data_path,f"{c}.png")), ImageReadMode.RGB) for c in examples[image_column]]
    examples["pixel_values"] = [image_transformations(image) for image in images]
    return examples


train_dataset.set_transform(transform_images)
val_dataset.set_transform(transform_images)
test_dataset.set_transform(transform_images)

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    input_ids = torch.tensor([example["input_ids"] for example in examples], dtype=torch.long)
    attention_mask = torch.tensor([example["attention_mask"] for example in examples], dtype=torch.long)
    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "return_loss": True,
    }

from sklearn.metrics import precision_recall_fscore_support


# def compute_metrics(pred):
#     labels = pred.label_ids
#     preds = pred.predictions.argmax(-1)
#     precision, _, _, _ = precision_recall_fscore_support(labels, preds, average='weighted')
#     return {
#         'precision': precision,
#     }

trainer = Trainer(
    model=model,
    args=TrainingArguments(output_dir="./checkpoints",
                           weight_decay=0.1,
                           dataloader_num_workers=0,
                           per_device_eval_batch_size=8,
                           per_device_train_batch_size=8,
                           num_train_epochs=4.0,
                           warmup_steps=0,
                           learning_rate=5e-05,
                           report_to="wandb"
                           ),
#     compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collate_fn
)


loading configuration file https://huggingface.co/openai/clip-vit-base-patch32/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3b789e359eb706935804a4c9db0b23519332fa3e1fad47b153db4fa81203a261.050ce7e9cbf1d6b484dcac0a3f0777b408c22a3e8bd14260000acef5fbc88aee
text_config_dict is None. Initializing the CLIPTextConfig with default values.
vision_config_dict is None. initializing the CLIPVisionConfig with default values.
Model config CLIPConfig {
  "_name_or_path": "openai/clip-vit-base-patch32",
  "architectures": [
    "CLIPModel"
  ],
  "initializer_factor": 1.0,
  "logit_scale_init_value": 2.6592,
  "model_type": "clip",
  "projection_dim": 512,
  "text_config": {
    "_name_or_path": "",
    "add_cross_attention": false,
    "architectures": null,
    "attention_dropout": 0.0,
    "bad_words_ids": null,
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "diversity_penalty"

Running tokenizer on train dataset:   0%|          | 0/15 [00:00<?, ?ba/s]

Running tokenizer on val dataset:   0%|          | 0/3 [00:00<?, ?ba/s]

Running tokenizer on test dataset:   0%|          | 0/4 [00:00<?, ?ba/s]

PyTorch: setting up devices


## 4. Train the Model

In [48]:
from transformers.trainer_utils import get_last_checkpoint
train_result = trainer.train(resume_from_checkpoint=get_last_checkpoint("./checkpoints"))


Loading model from ./checkpoints/checkpoint-7000.
/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14944
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7472
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 3
  Continuing training from global step 7000
  Will skip the first 3 epochs then the first 1396 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/1396 [00:00<?, ?it/s]

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




In [50]:
trainer.save_model()
trainer.log_metrics("train", train_result.metrics)
trainer.save_metrics("train", train_result.metrics)
trainer.save_state()

Saving model checkpoint to ./checkpoints
Configuration saved in ./checkpoints/config.json
Model weights saved in ./checkpoints/pytorch_model.bin


In [ ]:
# metrics = trainer.evaluate()
# trainer.log_metrics("eval", metrics)
# trainer.save_metrics("eval", metrics)

In [ ]:
kwargs = {
    "finetuned_from": "emoji-predictor", 
    "tasks": "contrastive-image-text-modeling", 
    "dataset": "AlekseyDorkin/extended_tweet_emojis"
}
trainer.push_to_hub(**kwargs)

## 5. Evaluate

In [37]:
from PIL import Image

dataset = load_dataset("vincentclaes/emoji-predictor")
test_dataset = dataset["test"].select(list(range(100)))
no_features = list(set(test_dataset["label"])) # these are the emoji's we use
emojis_pixel_values = transform_images({image_column: no_features})


test_dataset = test_dataset.map(
    function=tokenize_captions,
    batched=True,
    remove_columns=[col for col in column_names if col != image_column],
    num_proc=None,
    load_from_cache_file=False,
    desc="Running tokenizer on test dataset",
)
test_dataset.set_transform(transform_images)
test_dataset

Using custom data configuration vincentclaes--emoji-predictor-84ee9ecf6ec78809
Reusing dataset parquet (/root/.cache/huggingface/datasets/vincentclaes___parquet/vincentclaes--emoji-predictor-84ee9ecf6ec78809/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

Running tokenizer on test dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 100
})

In [38]:
trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 100
  Batch size = 8


AttributeError: 'BaseModelOutputWithPooling' object has no attribute 'detach'

In [18]:
x = collate_fn(test_dataset)


In [28]:
inputs = {"input_ids": x["input_ids"], "attention_mask" : x["attention_mask"], "pixel_values": emojis_pixel_values["pixel_values"]}


In [31]:
model = CLIPModel.from_pretrained("checkpoints/checkpoint-7000").to("cuda")
outputs = model(**x.to("cuda"))

loading configuration file checkpoints/checkpoint-7000/config.json
text_config_dict is None. Initializing the CLIPTextConfig with default values.
vision_config_dict is None. initializing the CLIPVisionConfig with default values.
Model config CLIPConfig {
  "_name_or_path": "openai/clip-vit-base-patch32",
  "architectures": [
    "CLIPModel"
  ],
  "initializer_factor": 1.0,
  "logit_scale_init_value": 2.6592,
  "model_type": "clip",
  "projection_dim": 512,
  "text_config": {
    "_name_or_path": "",
    "add_cross_attention": false,
    "architectures": null,
    "attention_dropout": 0.0,
    "bad_words_ids": null,
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "dropout": 0.0,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": 2,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
  

AttributeError: 'dict' object has no attribute 'to'

In [ ]:

CHECKPOINT = "checkpoints/checkpoint-4500"
# PROCESSOR = "openai/clip-vit-base-patch32"
predictions = []
references = []
SHARDS = 20

# not fully understanding this, but with torch.no_grad
# our GPU does not run out of memory.
with torch.no_grad():
    for i in range(SHARDS):
        chunk = test_dataset.shard(num_shards=SHARDS, index=i)
#         chunk_text = chunk["text"]
#         chunk_label = chunk["label"]
#         model = CLIPModel.from_pretrained(CHECKPOINT).to("cuda")

#         chunk_text = chunk_text.map(
#             function=tokenize_captions,
#             batched=True,
#             remove_columns=[col for col in column_names if col != image_column],
#             num_proc=None,
#             load_from_cache_file=False,
#             desc="Running tokenizer on test dataset",
#         )
#         test_dataset.set_transform(transform_images)
#         print(chunk)
        inputs = {**chunk.to_pandas().to_dict(), **emojis_pixel_values}
#         collate_fn()
#         def collate_fn(examples):
#             pixel_values = torch.stack([example["pixel_values"] for example in examples])
#             input_ids = torch.tensor([example["input_ids"] for example in examples], dtype=torch.long)
#             attention_mask = torch.tensor([example["attention_mask"] for example in examples], dtype=torch.long)
#             return {
#                 "pixel_values": pixel_values,
#                 "input_ids": input_ids,
#                 "attention_mask": attention_mask,
#                 "return_loss": True,
#             }
#         outputs = model(**inputs.to("cuda"))
#         # we want the probability for each emoji per sentence.
#         logits_per_text = outputs.logits_per_text
#         # we take the softmax to get the label probabilities
#         probs = logits_per_text.softmax(dim=1)
#         predictions_for_chunk = [torch.argmax(prob).item() for prob in probs]
#         predictions.extend(predictions_for_chunk)
#         references.extend(chunk_label)
#         print(f"total predictions so far: {len(predictions)}")
#         torch.cuda.empty_cache()

In [ ]:
inputs["input_ids"][0]

In [22]:
trainer.predict(test_dataset)

The following columns in the test set don't have a corresponding argument in `CLIPModel.forward` and have been ignored: text. If text are not expected by `CLIPModel.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 100
  Batch size = 8


AttributeError: 'BaseModelOutputWithPooling' object has no attribute 'detach'